In [2]:
import os
import dask.dataframe as dd
import datetime as dt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Ouverture des bases de données
df_cct = pd.read_parquet("/home/onyxia/work/data/cct.parquet")


In [4]:
#Un peu de nettoyage

df_cct["Amount"] = df_cct["Amount"].str.replace("$","")
df_cct["Amount"] = df_cct["Amount"].astype(float)


## Variables delta_t

In [5]:
#Création des variables 'timestamp', 'delta_t_s' et 'delta_t_s_card'
def time(dataframe):
   dataframe[['Hours', 'Minutes']] = dataframe.Time.str.split(":", n=1, expand=True) #scinder la colonne Time en 2
   dataframe["Hours"] = dataframe["Hours"].astype(int)
   dataframe["Minutes"] = dataframe["Minutes"].astype(int)
   #Créer une colonne 'Timestamp' : format année, mois, jour, heure, minute:
   timestamp = pd.to_datetime(dataframe[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
   date = pd.to_datetime(dataframe[["Year",'Month','Day']], format = '%Y:%M:%D%')
   dataframe["date"] = date.values
   dataframe["timestamp"] = timestamp.values
   dataframe["delta_t_card"] = (timestamp-timestamp.shift()) #calculer l'intervalle de temps entre
   #deux transactions de la même carte et du même user

   #Convertir delta_t_card en secondes et mettre à 0 les temps négatifs 
   #(ici, dataframe ordonné en fonction de l'user et de la carte, et non de la chronologie):
   dataframe["delta_t_s_card"] = dataframe["delta_t_card"].dt.total_seconds().fillna(0).astype(int)
   dataframe["delta_t_s_card"] = dataframe["delta_t_s_card"].mask(dataframe["delta_t_s_card"] < 0, 0)

   #Réordonner les transactions en fonction du User et de la chronoogie de ses transactions : 
   dataframe = dataframe.sort_values(["User", "Year","Month","Day","Hours","Minutes"], ascending=[True,True,True,True,True,True]).reset_index(drop=True)
   timestamp2 = pd.to_datetime(dataframe[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
   dataframe['delta_t'] = (timestamp2-timestamp2.shift()) #différence de temps entre deux transactions du même user
   dataframe["delta_t_s"] = dataframe["delta_t"].dt.total_seconds().fillna(0).astype(int) #pour la première
   #transaction, fixer le delta_t_s à 0
   dataframe = dataframe[['User', 'Card', 'date', 'timestamp', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?', 'delta_t_s','delta_t_s_card']] #ordonner et sélectionner les colonnes
   
   return dataframe




In [6]:
df_cct = time(df_cct)

## Variables journalières

In [7]:
#Determining the daily amount of transactions
day_amt_v = df_cct.groupby(["User", "date"])["Amount"].sum().values
day_amt_i = df_cct.groupby(["User", "date"])["Amount"].sum().index
#Determining the daily number of transactions
day_nbt_v = df_cct.groupby(["User", "date"])["Amount"].size().values
day_nbt_i = df_cct.groupby(["User", "date"])["Amount"].size().index

df1 = pd.DataFrame(day_amt_v,day_amt_i,columns=["day_amt"]).reset_index()
df2 = pd.DataFrame(day_nbt_v,day_nbt_i,columns=["day_nbt"]).reset_index()


df1["day_nbt"] = df2["day_nbt"]
df1["day_med_nbt"] = df1["day_nbt"].expanding().median()



In [8]:
df_cct = df_cct.merge(df1,on=["User","date"])

## Variables hebdomadaires

In [9]:
#df_cct.groupby(["User", pd.Grouper(key="date",freq="1W")]).Amount.sum()

In [10]:
df = pd.DataFrame()
#Création des variables:
#wk_i_nbt: nombre de transactions hebdomadaires (jour 1: première transaction de chaque user)
#wk_med_nbt: nombre médian de transactions hebdomadaires (jusqu'à la semaine i)
#wk_i_amt: nmontant total des transactions hebdomadaires 
#wk_med_amt: montant médian des transactions hebdomadaires (jusqu'à la semaine i)

for i in range(len(df_cct["User"].unique())):
    df3 = pd.DataFrame(df_cct[df_cct["User"] == i][['User',"date"]])
    wk_nbt_i = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")]).size().index
    wk_nbt_v = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")]).size().values


    wk_amt_i = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")])["Amount"].sum().index
    wk_amt_v = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")])["Amount"].sum().values

    wk_amt_med_i = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")])["Amount"].median().index
    wk_amt_med_v = df_cct[df_cct["User"] == i].groupby([pd.Grouper(key="date", freq="7d")])["Amount"].median().values

    df4 = pd.DataFrame(wk_nbt_v,wk_nbt_i, columns=["wk_i_nbt"]).reset_index()
    df4["User"] = i
    df4["wk_med_nbt"] = df4["wk_i_nbt"].expanding().median()
    df4["wk_i_amt"] = wk_amt_v
    df4["wk_med_amt"] = df4["wk_i_amt"].expanding().median()
    df4["wk_i_med_amt"] = wk_amt_med_v
 
    df5 = pd.merge(df3,df4,how='left',on=["User","date"])
    df = pd.concat([df,df5], axis=0)
    df = df[['wk_i_nbt', 'wk_med_nbt', 'wk_i_amt',
       'wk_med_amt','wk_i_med_amt']].fillna(method="ffill")
    

df.reset_index(drop=True, inplace=True) #dataframe avec les 4 nouvelles var. hebdomdaires

In [11]:
df["nbt_wk_i/amt_wk_i"] = df["wk_i_nbt"]/df["wk_i_med_amt"]

In [12]:
df_cct = pd.concat([df_cct,df[['wk_i_nbt', 'wk_med_nbt', 'wk_i_amt',
       'wk_med_amt',"wk_i_med_amt","nbt_wk_i/amt_wk_i"]]], axis=1)

## Variable montant/revenu journalier

In [13]:
import numpy as np

In [14]:
df_users = pd.read_csv("/home/onyxia/work/data/sd254_users.csv")

In [15]:
df_users['Yearly Income - Person'] = df_users['Yearly Income - Person'].str.replace("$","")
df_users["day_income"] = df_users['Yearly Income - Person'].astype(np.int)/365

In [16]:
df_users.reset_index(inplace=True)

In [17]:
df_users.rename(columns={"index":"User"},inplace=True)

In [18]:
df_cct["amt/day_income"] = df_cct["Amount"]/pd.merge(df_cct,df_users[["User","day_income"]], on="User",how="left")["day_income"]

In [19]:
df_cct

,User,Card,date,timestamp,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,...,day_amt,day_nbt,day_med_nbt,wk_i_nbt,wk_med_nbt,wk_i_amt,wk_med_amt,wk_i_med_amt,nbt_wk_i/amt_wk_i,amt/day_income
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,...,172.57,2,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.819868
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,172.57,2,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.235279
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,249.29,2,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.735796
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,...,249.29,2,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.788441
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,...,190.90,2,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.640230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020-02-27,2020-02-27 22:23:00,-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,315.71,8,3.0,39.0,32.0,1993.11,1859.19,54.000,0.722222,-0.299049
24386896,1999,1,2020-02-27,2020-02-27 22:24:00,54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,315.71,8,3.0,39.0,32.0,1993.11,1859.19,54.000,0.722222,0.299049
24386897,1999,1,2020-02-28,2020-02-28 07:43:00,59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,147.40,3,3.0,3.0,32.0,147.40,1852.71,45.130,0.066475,0.327569
24386898,1999,1,2020-02-28,2020-02-28 20:10:00,43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,147.40,3,3.0,3.0,32.0,147.40,1852.71,45.130,0.066475,0.238796


## Variable : lieu de transaction == ou != lieu de résidence du user (hors transaction en ligne)

In [20]:
df_cct = pd.merge(df_cct, df_users[["User","State"]], on="User", how="left") #temporary column

# Variable sur la transaction
df_cct["merchant_user_state_is_diff"] = 0
df_cct.loc[df_cct["Merchant State"] != df_cct["State"]]["merchant_user_state_is_diff"] = 1
df_cct.loc[df_cct["Merchant State"] == "Online Transaction"]["merchant_user_state_is_diff"] = np.nan
df_cct

,User,Card,date,timestamp,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,...,day_med_nbt,wk_i_nbt,wk_med_nbt,wk_i_amt,wk_med_amt,wk_i_med_amt,nbt_wk_i/amt_wk_i,amt/day_income,State,merchant_user_state_is_diff
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,...,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.819868,CA,0
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.235279,CA,0
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.735796,CA,0
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,...,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.788441,CA,0
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,...,2.0,16.0,16.0,1380.96,1380.96,90.015,0.177748,0.640230,CA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020-02-27,2020-02-27 22:23:00,-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,3.0,39.0,32.0,1993.11,1859.19,54.000,0.722222,-0.299049,NH,0
24386896,1999,1,2020-02-27,2020-02-27 22:24:00,54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,3.0,39.0,32.0,1993.11,1859.19,54.000,0.722222,0.299049,NH,0
24386897,1999,1,2020-02-28,2020-02-28 07:43:00,59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,3.0,3.0,32.0,147.40,1852.71,45.130,0.066475,0.327569,NH,0
24386898,1999,1,2020-02-28,2020-02-28 20:10:00,43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,3.0,3.0,32.0,147.40,1852.71,45.130,0.066475,0.238796,NH,0


In [21]:
df_cct.drop(["State"], axis = 1, inplace=True)

In [26]:
df_cct.to_parquet("/home/onyxia/work/data/cct_var.parquet")

## Variables glissantes : antécédant de fraude, nb de transactions, montant cumulé, médiane du montant des transacions sur 7 jours...

In [1]:
import pandas as pd

In [44]:
df_cct1 = pd.read_parquet("/home/onyxia/work/data/cct_var.parquet")

In [45]:
df_cct1["Is Fraud?"].mask(df_cct1["Is Fraud?"] == "Yes",1 , inplace = True)
df_cct1["Is Fraud?"].mask(df_cct1["Is Fraud?"] == "No", 0 , inplace = True)

In [46]:
def calc_rolling_nbt(dataf, column=None, setting='7D'):
    return (dataf
            .groupby('User')[column]
            .transform(lambda d: d.rolling(setting, min_periods=1).count()))

In [47]:
def calc_rolling_sum(dataf, column=None, setting='7D'):
    return (dataf
            .groupby('User')[column]
            .transform(lambda d: d.rolling(setting, min_periods=1).sum()))

In [48]:
def calc_rolling_med(dataf, column=None, setting='7D'):
    return (dataf
            .groupby('User')[column]
            .transform(lambda d: d.rolling(setting, min_periods=1).median()))

In [49]:
df_cct1["rolling_nb_frauds_wk"] = (df_cct1
.set_index('date')
.assign(rolling_nb_frauds_wk=lambda d: calc_rolling_sum(d, column='Is Fraud?')))["rolling_nb_frauds_wk"].values

In [51]:
df_cct1["rolling_nbt_wk"] = (df_cct1
.set_index('date')
.assign(rolling_nbt_wk=lambda d: calc_rolling_nbt(d, column= "Amount")))["rolling_nbt_wk"].values

In [52]:
df_cct1["rolling_amt_wk"] = (df_cct1
.set_index('date')
.assign(rolling_amt_wk=lambda d: calc_rolling_sum(d, column='Amount')))["rolling_amt_wk"].values

In [53]:
df_cct1["rolling_amt_med_wk"] = (df_cct1
.set_index('date')
.assign(rolling_amt_med_wk=lambda d: calc_rolling_med(d, column='Amount')))["rolling_amt_med_wk"].values

In [55]:
df_cct1["rolling_nbt_wk/amt_wk"] = df_cct1["rolling_nbt_wk"]/df_cct1["rolling_amt_wk"]

In [56]:
df_cct1

,User,Card,date,timestamp,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,...,wk_med_amt,wk_i_med_amt,nbt_wk_i/amt_wk_i,amt/day_income,merchant_user_state_is_diff,rolling_nb_frauds_wk,rolling_nbt_wk,rolling_amt_wk,rolling_amt_med_wk,rolling_nbt_wk/amt_wk
0,0,0,2002-09-01,2002-09-01 06:21:00,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,...,1380.96,90.015,0.177748,0.819868,0,0.0,1.0,134.09,134.090,0.007458
1,0,0,2002-09-01,2002-09-01 06:42:00,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,1380.96,90.015,0.177748,0.235279,0,0.0,2.0,172.57,86.285,0.011589
2,0,0,2002-09-02,2002-09-02 06:22:00,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,...,1380.96,90.015,0.177748,0.735796,0,0.0,3.0,292.91,120.340,0.010242
3,0,0,2002-09-02,2002-09-02 17:45:00,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,...,1380.96,90.015,0.177748,0.788441,0,0.0,4.0,421.86,124.645,0.009482
4,0,0,2002-09-03,2002-09-03 06:23:00,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,...,1380.96,90.015,0.177748,0.640230,0,0.0,5.0,526.57,120.340,0.009495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020-02-27,2020-02-27 22:23:00,-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,1859.19,54.000,0.722222,-0.299049,0,0.0,38.0,1939.11,52.210,0.019597
24386896,1999,1,2020-02-27,2020-02-27 22:24:00,54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,...,1859.19,54.000,0.722222,0.299049,0,0.0,39.0,1993.11,54.000,0.019567
24386897,1999,1,2020-02-28,2020-02-28 07:43:00,59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,1852.71,45.130,0.066475,0.327569,0,0.0,31.0,1610.29,54.000,0.019251
24386898,1999,1,2020-02-28,2020-02-28 20:10:00,43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,...,1852.71,45.130,0.066475,0.238796,0,0.0,32.0,1653.41,52.160,0.019354


In [ ]:
df_cct1.to_parquet("/home/onyxia/work/data/cct_var_new.parquet")